# Trading Performance

This notebook retrieves your trade data from Binance to calculate the pnl from your trading.

The methodology for calculating PNL is:

1. Calculate the total amount of net base asset purchased (or sold) and the total amount of quote asset spent (or received)
2. Calculate the value of these changes based on current asset spot price
3. Calculate the value of the fees paid out in the trades

Note that this methodology is one way of trying to evaluate the benefit if your trading activity, i.e. doing something versus doing nothing.  This does not capture any changes in portfolio value due to general market movements that may result in the appreciation of the value of base assets and quote assets.

## Instructions

Step 0) Select `Runtime` => `Run all`

Step 1) Input your ***read only*** API keys

Step 2) Input trading pair and start date

Step 3) click start

## Notes

- `% gain and loss` is based on your current balance of base and quote asset.  This may not be a comprehensive figure if (1) you have made deposits/withdrawals within the period being analyzed, and (2) if you are trading multiple pairs with overlapping base and quote assets

## Comments / bugs / suggestions

Please email [carlo@hummingbot.io](mailto:carlol@hummingbot.io?subject=Colab:%20Performance%20Sheet).  
Please email [amine@hummingbot.io](mailto:amine@hummingbot.io?subject=Colab:%20Performance%20Sheet).

## Input

In [9]:
#@title
#pull the logic code for colab env
from ipywidgets import Layout, Button, Box, FloatText,Text, Password, Dropdown, Label, IntSlider,DatePicker
from IPython.display import clear_output
import datetime

exchange_in = Dropdown(options=['binance', 'kucoin', 'ascendex'])
base_asset_in = Text()
quote_asset_in = Text()
api_key_in = Password()
api_secret_in = Password()
api_passphrase_in = Password()
api_group_in = Text()
start_date_in = DatePicker(disabled=False,value=datetime.datetime.now())
end_date_in = DatePicker(disabled=False,value=datetime.datetime.now())

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_passphrase_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_group_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

box_passPhrase = Box([Label(value='Passphrase'), 
    api_passphrase_in], layout=op_item_passphrase_layout)
box_group =   Box([Label(value='Api group'), 
    api_group_in], layout=op_item_group_layout)

form_items = [
    Box([Label(value='Exchange'), exchange_in], layout=form_item_layout),
    Box([Label(value='Trading pair'), Box([base_asset_in,Label(value='/'),quote_asset_in])], layout=form_item_layout),
    Box([Label(value='Api Key'), 
         api_key_in], layout=form_item_layout),
    Box([Label(value='Secret Key'), 
         api_secret_in], layout=form_item_layout),
    box_passPhrase,
    box_group,
    Box([Label(value='Start Date'), start_date_in], layout=form_item_layout),
    Box([Label(value='End Date'), end_date_in], layout=form_item_layout),
]

button = Button(
    description='Start !',
    button_style='success'
  )

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

box_passPhrase.layout.display = 'none'
box_group.layout.display = 'none'
display(form,button)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if(change['new'] == 'kucoin'):
            box_passPhrase.layout.display = 'flex'
            box_group.layout.display = 'none'
        elif(change['new'] == 'ascendex'):
            box_passPhrase.layout.display = 'none'
            box_group.layout.display = 'flex'
        else:
            box_passPhrase.layout.display = 'none'
            box_group.layout.display = 'none'


exchange_in.observe(on_change)

def start(b):    
    clear_output(wait=True)
    if 'google.colab' in str(get_ipython()):
      !git clone https://github.com/CoinAlpha/pnl-analysis
      %cd pnl-analysis/
    clear_output(wait=True)
    import pandas as pd
    import numpy as np
    import json
    import plotly.graph_objects as go
    from IPython.core.display import display, HTML
    from datetime import datetime
    from src.processing import pnl_calculate

    # Set display
    pd.options.display.float_format = '{:,.10f}'.format

    import warnings
    warnings.filterwarnings("ignore")

    START_TIME=start_date_in.value
    END_TIME=end_date_in.value
    exchange = exchange_in.value
    api_key = api_key_in.value
    api_secret = api_secret_in.value
    api_passphrase = api_passphrase_in.value
    api_group = api_group_in.value
    client = None
    trading_pair = None   
    start_dt = int(pd.to_datetime(START_TIME).timestamp()*1000)
    end_dt = int(pd.to_datetime(END_TIME).timestamp()*1000)
    if exchange == 'binance': 
        !pip install binance
        !pip install python-binance
        from src.binance.BinanceClientWrapper import BinanceClientWrapper 
        client = BinanceClientWrapper.createInstance(api_key,api_secret)
        trading_pair = base_asset_in.value+quote_asset_in.value   
    elif exchange == 'kucoin':
        !pip install kucoin-python
        from src.kucoin.KucoinClientWrapper import KucoinClientWrapper
        client = KucoinClientWrapper.createInstance(api_key,api_secret,api_passphrase)
        trading_pair = base_asset_in.value+"-"+quote_asset_in.value   

    elif exchange == 'ascendex':
        from src.ascendex.AscendexClientWrapper import AscendexClientWrapper
        client = AscendexClientWrapper.createInstance(api_key,api_secret,api_group)
        trading_pair = base_asset_in.value+"/"+quote_asset_in.value   

    from jinja2 import Template

    class Printer:
        @staticmethod
        def h1(*title):
            title_t = Template("<h1>{{title}}</h1>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h2(*title):
            title_t = Template("<h2>{{title}}</h2>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h3(*title):
            title_t = Template("<h3>{{title}}</h3>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))

        @staticmethod
        def html(html):
            display(HTML(html))
        
        @staticmethod
        def p_df(df):
            """Neatly display a dataframe"""
            display(HTML(df.to_html()))
            
        def p_dict(dt):
            df = pd.DataFrame()
            df['_'] = dt.keys()
            df['__'] = dt.values()
            Printer.p_df(df)

    class Vis:
        
        def graph_trades(df):
            df = df.pivot_table(values=["qty"], columns=["side"], index=["date_time"], aggfunc=np.sum, fill_value=0)
            df.columns = map(lambda x: x[1], df.columns)
            df = df.resample("h").sum()
            if 'sell' in df:
                df["sell"] = df["sell"] * -1

            fig = go.Figure()
            x = df.index
            for name in np.sort(df.columns):
                y = df[name]
                fig.add_trace(go.Bar(x=x, y=y, name=name))
            fig.update_layout(barmode="relative", legend_orientation="h", yaxis_tickformat=",.0f", yaxis_title="Base token amounts")
            fig.show()

    balance,base,quote,base_price,quote_price = client.get_current_asset_balance(trading_pair)
    Printer.h3(f"Loading Trades {trading_pair} from {START_TIME} to {END_TIME} : ")
    Printer.html("<img src=https://i.imgur.com/CS6l6tB.gif>")
    df_trades = client.get_trades(trading_pair,start_dt,end_dt)
    meta = {
        'base_asset':base,
        'quote_asset':quote,
        'quote_asset_price':quote_price,
        'base_asset_price':base_price
    }
    clear_output(wait=True)
    display(form,button)
    summary,df_summary_table,total_fees_usd,df_commissions = pnl_calculate(df_trades,balance,meta)
    Printer.h1(trading_pair,":",START_TIME,"to", END_TIME)

    total_balance_usd = balance['usd_value'].sum()
    Printer.h3(f"Current balance: ${total_balance_usd:,.2f}")
    Printer.p_df(balance)

    ## Calculate performance
    Printer.h2("Trades")
    Printer.p_df(df_trades)
    Printer.h2("Performance summary")
    Printer.p_dict(summary)
    Printer.p_df(df_summary_table)
    Printer.h2(f"Trade commissions: {total_fees_usd}")
    Printer.p_df(df_commissions)
    Printer.h3("Historical trades")
    Vis.graph_trades(df_trades)

button.on_click(start)

Box(children=(Box(children=(Label(value='Exchange'), Dropdown(options=('binance', 'kucoin', 'ascendex'), value…

Button(button_style='success', description='Start !', style=ButtonStyle())